In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
from branca.colormap import linear

In [2]:
gdf = gpd.read_file('russia.geojson')

In [2]:
file = 'clusters.xlsx'
df = pd.read_excel(file)
df

,Number,Class,Sex,Rus,Math,Family,Region,History Knowledge Index,Family Life Index,Culture Index,Cluster
0,1964895,6,Девочки,3,3,Полная-немногодетная,Северо-Западный федеральный округ,2,2,2,5
1,1964302,6,Девочки,5,5,Многодетная-полная,Северо-Западный федеральный округ,2,2,2,6
2,1965852,6,Мальчики,4,4,Полная-немногодетная,Северо-Западный федеральный округ,3,3,3,4
3,1953331,6,Мальчики,3,3,Полная-немногодетная,Северо-Западный федеральный округ,2,2,2,5
4,1964793,6,Девочки,5,4,Полная-немногодетная,Северо-Западный федеральный округ,3,3,3,4
...,...,...,...,...,...,...,...,...,...,...,...
39255,2076893,8,Мальчики,4,4,Полная-немногодетная,Сибирский федеральный округ,3,3,3,4
39256,2077118,8,Девочки,4,5,Полная-немногодетная,Сибирский федеральный округ,2,2,2,6
39257,2077124,8,Мальчики,3,3,Полная-немногодетная,Сибирский федеральный округ,2,2,2,5
39258,2077150,8,Девочки,4,3,Полная-немногодетная,Сибирский федеральный округ,2,3,2,3


In [5]:
file_path = 'NIKO_data.xlsx'
d2f = pd.read_excel(file_path)
merged_df = pd.merge(df, d2f[['Number', 'R']], on='Number', how='inner')
merged_df

,Number,Class,Sex,Rus,Math,Family,Region,History Knowledge Index,Family Life Index,Culture Index,Cluster,R
0,1964895,6,Девочки,3,3,Полная-немногодетная,Северо-Западный федеральный округ,2,2,2,5,г. Санкт-Петербург
1,1964302,6,Девочки,5,5,Многодетная-полная,Северо-Западный федеральный округ,2,2,2,6,г. Санкт-Петербург
2,1965852,6,Мальчики,4,4,Полная-немногодетная,Северо-Западный федеральный округ,3,3,3,4,г. Санкт-Петербург
3,1953331,6,Мальчики,3,3,Полная-немногодетная,Северо-Западный федеральный округ,2,2,2,5,г. Санкт-Петербург
4,1964793,6,Девочки,5,4,Полная-немногодетная,Северо-Западный федеральный округ,3,3,3,4,г. Санкт-Петербург
...,...,...,...,...,...,...,...,...,...,...,...,...
39425,2076893,8,Мальчики,4,4,Полная-немногодетная,Сибирский федеральный округ,3,3,3,4,Новосибирская область
39426,2077118,8,Девочки,4,5,Полная-немногодетная,Сибирский федеральный округ,2,2,2,6,Новосибирская область
39427,2077124,8,Мальчики,3,3,Полная-немногодетная,Сибирский федеральный округ,2,2,2,5,Новосибирская область
39428,2077150,8,Девочки,4,3,Полная-немногодетная,Сибирский федеральный округ,2,3,2,3,Новосибирская область


In [6]:
df_cluster = merged_df.copy()
objects_by_region = df_cluster['R'].value_counts().reset_index()
objects_by_region.columns = ['R', 'object_count']
df_cluster = pd.merge(df_cluster, objects_by_region, on='R', how='left')
df_cluster = df_cluster[["Cluster", "R", "object_count"]]
df_cluster

,Cluster,R,object_count
0,5,г. Санкт-Петербург,2016
1,6,г. Санкт-Петербург,2016
2,4,г. Санкт-Петербург,2016
3,5,г. Санкт-Петербург,2016
4,4,г. Санкт-Петербург,2016
...,...,...,...
39425,4,Новосибирская область,1736
39426,6,Новосибирская область,1736
39427,5,Новосибирская область,1736
39428,3,Новосибирская область,1736


In [9]:
gdf_regions = gdf.merge(objects_by_region, how="left", left_on='region', right_on='R')
gdf_regions['object_count'] = gdf_regions['object_count'].fillna(0)
gdf_regions

,region,geometry,R,object_count
0,Алтайский край,"POLYGON ((82.80364 50.94067, 82.78856 50.94470...",NaN,0.0
1,Амурская область,"POLYGON ((131.38809 49.25325, 131.28198 49.250...",Амурская область,212.0
2,Архангельская область,"MULTIPOLYGON (((36.77240 63.91510, 36.77083 63...",NaN,0.0
3,Астраханская область,"POLYGON ((47.59436 45.44006, 47.58152 45.44518...",NaN,0.0
4,Белгородская область,"POLYGON ((38.93024 49.82358, 38.92225 49.83007...",Белгородская область,312.0
...,...,...,...,...
80,Ямало-Ненецкий автономный округ,"MULTIPOLYGON (((65.99583 69.06250, 65.99375 69...",Ямало-Ненецкий автономный округ,27.0
81,Ярославская область,"POLYGON ((38.32256 56.75940, 38.27213 56.80248...",Ярославская область,703.0
82,г. Москва,"MULTIPOLYGON (((37.24789 55.58891, 37.24775 55...",NaN,0.0
83,г. Санкт-Петербург,"MULTIPOLYGON (((29.71917 59.97542, 29.71917 59...",г. Санкт-Петербург,2016.0


In [10]:
latitude = 55
longitude = 37
m = folium.Map(location=[latitude, longitude], zoom_start=10)

In [11]:
heat_data = [(row.geometry.centroid.y, row.geometry.centroid.x, row.object_count) for idx, row in gdf_regions.iterrows() if row.object_count > 0]
HeatMap(heat_data).add_to(m)
m.save('heatmap_all.html')

In [12]:
latitude = 55
longitude = 37
for cluster in range(1, 7):
    m = folium.Map(location=[latitude, longitude], zoom_start=10)
    temp = (df_cluster.groupby(['Cluster', 'R']).count() / df_cluster.groupby('R').count())
    objects_by_region = temp.loc[cluster].reset_index()
    objects_by_region = objects_by_region.drop(columns=['Cluster'])
    objects_by_region['object_count'] = objects_by_region['object_count'] * 1000
    objects_by_region['object_count'] = objects_by_region['object_count'].round().astype(int)
    objects_by_region.columns = ['R', 'object_count']
    gdf_regions = gdf.merge(objects_by_region, how="left", left_on='region', right_on='R')
    gdf_regions['object_count'] = gdf_regions['object_count'].fillna(0)
    heat_data = [(row.geometry.centroid.y, row.geometry.centroid.x, row.object_count) for idx, row in gdf_regions.iterrows() if row.object_count > 0]
    HeatMap(heat_data).add_to(m)
    m.save(f'heatmap_{cluster}.html')

'\nlatitude = 55\nlongitude = 37\nfor cluster in range(1, 7):  # Изменили диапазон значений cluster\n    m = folium.Map(location=[latitude, longitude], zoom_start=10)\n    objects_by_region = df_cluster[df_cluster[\'Cluster\']==cluster][\'R\'].value_counts().reset_index()\n    objects_by_region.columns = [\'R\', \'object_count\']\n    gdf_regions = gdf.merge(objects_by_region, how="left", left_on=\'region\', right_on=\'R\')\n    gdf_regions[\'object_count\'] = gdf_regions[\'object_count\'].fillna(0)\n    \n    heat_data = [(row.geometry.centroid.y, row.geometry.centroid.x, row.object_count)\n                 for idx, row in gdf_regions.iterrows() if row.object_count > 0]\n    \n    HeatMap(heat_data).add_to(m)\n    \n    m.save(f\'heatmap_{cluster}.html\')\n    '